In [54]:
import $ivy.`org.apache.spark:spark-core_2.12:2.4.3`
import $ivy.`org.apache.spark:spark-sql_2.12:2.4.3`
import $ivy.`org.apache.spark:spark-catalyst_2.12:2.4.3`
import $ivy.`org.apache.spark:spark-streaming_2.12:2.4.3`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.12:2.4.3`
import $ivy.`io.delta:delta-core_2.12:0.3.0`
// import $ivy.`sh.almond::almond-spark:0.6.0`

import $ivy.$                                       

import $ivy.$                                      

import $ivy.$                                           

import $ivy.$                                            

import $ivy.$                                                 

import $ivy.$                               
// import $ivy.`sh.almond::almond-spark:0.6.0`

In [46]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
import io.delta.tables._

import scala.util.{Try,Success,Failure}
import java.io.File

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._

import org.apache.spark.sql.Row

import io.delta.tables._


import scala.util.{Try,Success,Failure}

import java.io.File

In [55]:
implicit class RichDF(val ds: org.apache.spark.sql.DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

In [4]:
Logger.getLogger("org").setLevel(Level.OFF)

In [9]:
val spark = SparkSession
                .builder()
                .config("spark.driver.memory","5g")
                .master("local[*]")
                .getOrCreate()

import spark.implicits._

spark: SparkSession = org.apache.spark.sql.SparkSession@3b10b47c
import spark.implicits._

In [50]:
val dirṔath = "/home/vitor/Documents/datasets/events"

if(!new File(dirṔath).exists()) {
    val tempDF = spark.createDataFrame(spark.sparkContext.emptyRDD[Row],new StructType(
        Array(
            new StructField("id",IntegerType),
            new StructField("name",StringType),
            new StructField("maxPeopleAllowed",IntegerType),
            new StructField("eventType",StringType),
            new StructField("location",StringType),
            new StructField("lastUpdate",TimestampType)
        )
    ))
    tempDF.write.format("delta").option("path","/home/vitor/Documents/datasets/events").save()
}

dirṔath: String = "/home/vitor/Documents/datasets/events"

## Streaming

In [56]:
val columns = Seq("id","name","maxPeopleAllowed","eventType","location","lastUpdate")
val delta = DeltaTable.forPath(dirṔath)
val stream = spark
                .readStream
                .format("kafka")
                .option("kafka.bootstrap.servers", "localhost:9092")
                .option("subscribe", "animes_topic")
                .load()

val parsedDF = stream
                .selectExpr("cast(value as String) as value")
                .select(json_tuple($"value",columns:_*))
                .toDF(columns:_*)
                .withColumn("id", $"id".cast(IntegerType))
                .withColumn("lastUpdate",to_timestamp($"lastUpdate"))
parsedDF
    .writeStream
    .format("delta")
    .outputMode("update")
    .foreachBatch((microBatchOutputDF: org.apache.spark.sql.DataFrame, batchId: Long) => {
        
         delta.as("orig")
            .merge(
                microBatchOutputDF.as("updt"),
                "orig.id = updt.id and orig.lastUpdate < updt.lastUpdate"
            )
            .whenMatched().updateAll()
            .whenNotMatched("updt.id is not null").insertAll()
            .execute()        

    }).start().awaitTermination()

: 